# Pedro - Short Queeze Predictor
---

### 1. Libraries Import

In [ ]:
import os
import joblib
import numpy as np
import pandas as pd
from pathlib import Path
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi

import xgboost as xgb
import tensorflow as tf
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from tensorflow.keras.models import load_model

### 2. Alpaca API Setup

In [ ]:
def create_alpaca_api():
    load_dotenv(dotenv_path="Alpaca.env")

    alpaca_api_key = os.getenv("ALPACA_API_KEY")
    alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")
    alpaca_api_base_url = "https://paper-api.alpaca.markets"

    api = tradeapi.REST(
        alpaca_api_key,
        alpaca_secret_key,
        alpaca_api_base_url,
        api_version="v2"
    )

    return api

In [ ]:
api = create_alpaca_api()

In [ ]:
# Get account information
account = api.get_account()

# Check if our account is restricted from trading.
if account.trading_blocked:
    print('Account is currently restricted from trading.')

# Print our current balance.
print('${} is available as buying power.'.format(account.buying_power))

### 3. Load Models

In [ ]:
def load_models(model_names, folder_path='Models'):
    """
    Loads trained models based on the given model names from the specified folder.
    """
    models = {}
    for name in model_names:
        # Load each model from its file in the 'Models' folder
        if name == "Neural_Network":
            models[name] = load_model(os.path.join(folder_path, f"{name.replace(' ', '_')}.h5"))
        else:
            models[name] = joblib.load(os.path.join(folder_path, f"{name.replace(' ', '_')}.joblib"))
        
    return models

print("Function 'load_models' defined.")

In [ ]:
# Load models
model_names = ["Gradient_Boosting_Classifier", "Support_Vector_Classifier", 
               "Random_Forest_Classifier", "Decision_Tree_Classifier", 
               "XGBoost_Classifier", "Neural_Network"]

models = load_models(model_names)

### 4. Load New Data

In [ ]:
new_data = new_data = pd.read_csv(
    Path("Resources/NewData.csv"), #We dont have the NewData (Live Data)
    parse_dates=True,
    infer_datetime_format=True
)

In [ ]:
new_data.head()

### 5. Preprocess the New Data

In [ ]:
# Load the StandardScaler and OneHotEncoder
scaler = joblib.load('Models/scaler.joblib') 
encoder = joblib.load('Models/encoder.joblib')

In [ ]:
# Define numerical and categorical features
numerical_features = ['Short % of Float', 'Market Cap', '% from 50 day MA']  # replace with your actual numerical features
categorical_features = ['Sector'] # replace with your actual categorical features

In [ ]:
# Apply StandardScaler to the numerical columns of new_data
new_data[numerical_features] = scaler.transform(new_data[numerical_features])

In [ ]:
# Apply OneHotEncoder to the categorical columns of new_data
encoded_data = encoder.transform(new_data[categorical_features])
encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(categorical_features), index=new_data.index)
numerical_df = new_data.drop(columns=categorical_features, axis=1)

In [ ]:
# Combine the numerical and encoded categorical data
encoded_new_data = pd.concat([numerical_df, encoded_df], axis=1)

In [ ]:
# Set signal variable
signal = 1

In [ ]:
# Create buy signal, num shares and ticker
if signal == 1:
    orderSide = "buy"
else:
    orderSide = "sell"

In [ ]:
# Set the ticket symbol and the number of shares to buy
ticker = "TSLA"
number_of_shares = 1

In [ ]:
# Make API call
prices = api.get_bars(ticker, "1Min").df

# Reorganize the DataFrame
prices = pd.concat([prices], axis=1, keys=["TSLA"])

# Get final closing price
limit_amount = prices["TSLA"]["close"][-1]

In [ ]:
# Submit order
api.submit_order(
    symbol="TSLA", 
    qty=number_of_shares, 
    side=orderSide, 
    time_in_force="gtc", 
    type="limit", 
    limit_price=limit_amount
)